# Data Cleaning

This is the process followed to go from the generated runs to a clean dataset

In [1]:
import pandas as pd
import config as c

In [2]:
data = pd.read_csv('data/runs/n_results.csv', index_col='run')
drug_data = pd.read_csv('data/runs/drug_data.csv')
#old_drug_data = pd.read_csv('data/runs/drug_data.csv')
#new_drug_data = pd.read_csv('data/runs/new_drug.csv')

/usr/local_rwth/sw/python/3.7.3/x86_64/extra/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We only use runs that were completed and for which we obtained a $ r^2 $ score

In [3]:
data = data[data['status']=='completed']
data = data[data['r_2_mean'].notnull()]

We turn the time taken into a time delta and the start time into a datetime

In [4]:
data['time'] = pd.to_timedelta(data['time'])
data['started'] = pd.to_datetime(data['started'])

We create a new column (`seconds`) that stores the seconds taken by a specific run

In [5]:
data['seconds'] = data['time'].apply(lambda x:x.seconds)

We create a new column (`datasets`) which contains the number of datasets used

In [6]:
data['datasets'] = data['gdsc']+data['ctrp']+data['ccle']

We drop runs that used no datasets

In [7]:
data = data[data['datasets']>0]

All runs before 2020-04-15 18:00:00 are corrupt so we drop them

In [8]:
data = data[data['started'] > pd.to_datetime('2020-04-15 18:00:00')]

We drop all runs with IC quality below -0.95

In [ ]:
data = data[data['Quality']>-0.95]

## Append

This method allows us to attach the models created using `train.py` to the results dataframe. For this train should be ran using Guild AI. Here the directory to guild should be defined on the config file.

In [2]:
from config import guild
def append(data, run):
    result = pd.DataFrame()
    run = [i for i in run if i not in data['run'].values]
    print(len(run))
    for a in run:
        f = subprocess.run("guild select " +str(a), shell=True, capture_output=True)
        d = pd.read_csv(guild+'runs/'+f.stdout.decode('utf-8')[:-1]+'/scores.csv').rename(columns={'Unnamed: 0':'Drug'})
        d['run'] = [a for i in range(0, d.shape[0])]
        result = pd.concat([result, d])
        run.remove(a)
        if len(run) % 20 == 0:
            new = pd.concat([data, result])
            print('Drug data length is {}'.format(len(new)))
            print('{} runs left'.format(len(run)))
            new.to_csv('full_drug_new.csv', index=False)
    new = pd.concat([data, result])
    print('Drug data length is {}'.format(len(new)))
    new.to_csv('full_drug_new.csv', index=False)
    return new

In [39]:
k = append(data, ndata.index)

30


/rwthfs/rz/cluster/home/kz035322/Thesis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
/rwthfs/rz/cluster/home/kz035322/Thesis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


Drug data length is 162600
20 runs left


/rwthfs/rz/cluster/home/kz035322/Thesis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Drug data length is 162687
